In [1]:
from google.colab import files
uploaded = files.upload()


Saving homer_bart.zip to homer_bart.zip


In [6]:
import zipfile
import os

zip_file_name = 'homer_bart.zip'


unzip_dir = 'unzipped_files'
os.makedirs(unzip_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


print(os.listdir(unzip_dir))



['Bart', 'Homer']


In [34]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [12]:
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

data_dir = '/content/Data'
train_dir = 'path_to_your_dataset/train'
test_dir = 'path_to_your_dataset/test'

def resize_split_and_move_files(class_name, target_size=(64, 64)):
    class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)

    # Resize images
    for img_name in images:
        img_path = os.path.join(class_dir, img_name)
        img = Image.open(img_path)
        img = img.resize(target_size, Image.ANTIALIAS)  # Resize image to target_size
        img.save(img_path)  # Overwrite original image with resized image

    # Split the data
    train_images, test_images = train_test_split(images, test_size=0.1, random_state=42)

    # Function to move files to the new directory
    def move_files(image_list, destination_dir):
        dest_class_dir = os.path.join(destination_dir, class_name)
        os.makedirs(dest_class_dir, exist_ok=True)
        for image in image_list:
            src_path = os.path.join(class_dir, image)
            dst_path = os.path.join(dest_class_dir, image)
            shutil.copy(src_path, dst_path)

    # Move the files
    move_files(train_images, train_dir)
    move_files(test_images, test_dir)

# Get the class names from the dataset directory
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Resize, split, and move files for each class
for class_name in classes:
    resize_split_and_move_files(class_name)



<ipython-input-12-218307c43c86>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)  # Resize image to target_size


In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_data_dir = '/content/final_dataset/train'
test_data_dir = '/content/final_dataset/test'

img_height, img_width = 64, 64
batch_size = 32


train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)




Found 242 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [38]:
model = Sequential()
model.add(Flatten(input_shape=(img_height, img_width, 3)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer= Adam(lr= 0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=20)


Epoch 1/20
8/8 [==============================] - 2s 91ms/step - loss: 4.2706 - accuracy: 0.5207
Epoch 2/20
8/8 [==============================] - 1s 86ms/step - loss: 3.7170 - accuracy: 0.4711
Epoch 3/20
8/8 [==============================] - 1s 90ms/step - loss: 2.4609 - accuracy: 0.5455
Epoch 4/20
8/8 [==============================] - 1s 91ms/step - loss: 1.6022 - accuracy: 0.5744
Epoch 5/20
8/8 [==============================] - 1s 86ms/step - loss: 1.0772 - accuracy: 0.5785
Epoch 6/20
8/8 [==============================] - 1s 87ms/step - loss: 0.7692 - accuracy: 0.5537
Epoch 7/20
8/8 [==============================] - 1s 90ms/step - loss: 0.6867 - accuracy: 0.5909
Epoch 8/20
8/8 [==============================] - 1s 89ms/step - loss: 0.6766 - accuracy: 0.5620
Epoch 9/20
8/8 [==============================] - 1s 86ms/step - loss: 0.6419 - accuracy: 0.6157
Epoch 10/20
8/8 [==============================] - 1s 92ms/step - loss: 0.6255 - accuracy: 0.6033
Epoch 11/20
8/8 [============

In [39]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


1/1 [==============================] - 0s 204ms/step - loss: 0.5215 - accuracy: 0.8519
Test accuracy: 0.8518518805503845
